## Imports

In [1]:
import energyflow as ef
import numpy as np

In [2]:
# get events to use throughout
events = np.random.rand(1000,25,3)

## Polynomial classes

### EFP
_representing individual EFPs_

In [3]:
# graphs are specified as a list of tuples
graph0 = [(0,1),(0,1),(0,2),(1,2),(2,3)]

# arbitrary objects can be used to label vertices
graph1 = [('atlas','cms'),('atlas','cms'),('atlas','lhcb'),('cms','lhcb'),('lhcb','alice')]

# let's make EFPs for these graphs
graphs = [graph0, graph1]
efps = [ef.EFP(g) for g in graphs]

# and see that they give the same result
result = [efp.compute(events[0]) for efp in efps]
print('result on event 0:', result[0])
print('difference between graph0 and graph1:', abs(result[0] - result[1]))

result on event 0: 0.0303572706162
difference between graph0 and graph1: 6.93889390391e-18


In [4]:
# batch_compute makes computation on many events simple
results = efps[0].batch_compute(events)
print('results shape:', results.shape)
print('result on event 0:', results[0])

results shape: (1000,)
result on event 0: 0.0303572706162


In [5]:
# each efp has a few useful properties
efp = efps[0]
print('graph:', efp.graph)
print('simple_graph:', efp.simple_graph)
print('n:', efp.n)
print('d:', efp.d)
print('c:', efp.c)

graph: [(0, 1), (0, 1), (0, 2), (1, 2), (2, 3)]
simple_graph: [(0, 1), (0, 2), (1, 2), (2, 3)]
n: 4
d: 5
c: 3


### EFPSet
_representing a collection of EFPs_

In [6]:
# initialize from installed default EFP file
efpset = ef.EFPSet('d<=5', verbose=True)

Originally Available EFPs:
  Prime: 23690
  Composite: 21540
  Total:  45230
Current Stored EFPs:
  Prime: 53
  Composite: 48
  Total:  101


In [7]:
# demonstrate computation on an event
result = efpset.compute(events[0])
print('result on event 0:')
print(result)

result on event 0:
[ 0.46629287  0.27685894  0.18474402  0.13381899  0.10295552  0.22423246
  0.13654605  0.09313386  0.08490567  0.06871634  0.0589447   0.10343913
  0.06208453  0.04208523  0.03710076  0.11100742  0.06914562  0.04806309
  0.04385362  0.10748071  0.06528142  0.06713362  0.04442863  0.04681713
  0.03956486  0.04164426  0.05090274  0.03035727  0.03127484  0.03154052
  0.05394     0.03390356  0.02430545  0.05643912  0.03585858  0.05305464
  0.03297436  0.0338994   0.03214995  0.05153672  0.03131099  0.03210652
  0.02575768  0.0249762   0.02434483  0.02656846  0.0251619   0.02939029
  0.02690626  0.02611848  0.02536456  0.02544765  0.02471071  0.21742904
  0.12909735  0.08614482  0.07665087  0.05114803  0.06239884  0.104558
  0.06208076  0.06367045  0.04823293  0.04142561  0.03780399  0.02863805
  0.04342765  0.03959091  0.02894957  0.10138561  0.05176197  0.05011749
  0.0502802   0.06019717  0.03224211  0.03044026  0.03130393  0.02373558
  0.02515184  0.0307334   0.029757

In [8]:
# demonstrate computation on many events
results = efpset.batch_compute(events)
print('results shape:', results.shape)
print('result on event 0:')
print(results[0])

results shape: (1000, 101)
result on event 0:
[ 0.46629287  0.27685894  0.18474402  0.13381899  0.10295552  0.22423246
  0.13654605  0.09313386  0.08490567  0.06871634  0.0589447   0.10343913
  0.06208453  0.04208523  0.03710076  0.11100742  0.06914562  0.04806309
  0.04385362  0.10748071  0.06528142  0.06713362  0.04442863  0.04681713
  0.03956486  0.04164426  0.05090274  0.03035727  0.03127484  0.03154052
  0.05394     0.03390356  0.02430545  0.05643912  0.03585858  0.05305464
  0.03297436  0.0338994   0.03214995  0.05153672  0.03131099  0.03210652
  0.02575768  0.0249762   0.02434483  0.02656846  0.0251619   0.02939029
  0.02690626  0.02611848  0.02536456  0.02544765  0.02471071  0.21742904
  0.12909735  0.08614482  0.07665087  0.05114803  0.06239884  0.104558
  0.06208076  0.06367045  0.04823293  0.04142561  0.03780399  0.02863805
  0.04342765  0.03959091  0.02894957  0.10138561  0.05176197  0.05011749
  0.0502802   0.06019717  0.03224211  0.03044026  0.03130393  0.02373558
  0.025

In [9]:
# batch_compute is faster than using compute repeatedly
%timeit -r 3 efpset.batch_compute(events)
%timeit -r 1 [efpset.compute(event) for event in events]

1.98 s ± 3.17 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
7.71 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
# zs and thetas can be passed in explicitly if you want to use a custom measure
zs = np.random.rand(100,25)
thetas = np.random.rand(100,25,25)
results = efpset.batch_compute(zs=zs, thetas=thetas)

In [11]:
# examine all the graphs in the EFPSet
graphs = efpset.graphs()

# examine only those graphs with chi == 3
chi3_graphs = efpset.graphs('c == 3')

## Additional features (`igraph` must be installed)

### Generator

In [12]:
# initialize from generator
print('Initializing from generator:')
gen = ef.Generator(dmax=7)
efpset0 = ef.EFPSet(gen, 'd<=6', verbose=True)

print()

# initialize from custom file
print('Initializing from custom file:')
gen.save('test')
efpset1 = ef.EFPSet('d<=6', filename='test', verbose=True)
import os
os.remove('test.npz')

Initializing from generator:
Originally Available EFPs:
  Prime: 489
  Composite: 510
  Total:  999
Current Stored EFPs:
  Prime: 156
  Composite: 157
  Total:  313

Initializing from custom file:
Originally Available EFPs:
  Prime: 489
  Composite: 510
  Total:  999
Current Stored EFPs:
  Prime: 156
  Composite: 157
  Total:  313
